In [ ]:
# in requirements.txt
# %pip install "zenml[server]"

In [8]:
#!zenml integration install sklearn -y

NumExpr defaulting to 2 threads.
⠴ Installing integrations...


In [9]:
#!pip install pyparsing==2.4.2

In [11]:
#import IPython

#IPython.Application.instance().kernel.do_shutdown(restart = True)

{'status': 'ok', 'restart': True}

In [1]:
import config
NGROK_TOKEN = config.NGROK_TOKEN

In [2]:
from zenml.environment import Environment

if Environment.in_google_colab() :
    #!pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

In [3]:
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.72.0
⠋ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
Creating database tables
⠙ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠹ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠸ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠼ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠴ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠦ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠧ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠇ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠏ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠋ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠙ Initializing ZenML repository at /workspaces/MLOps-using-ZenML/notebooks.
⠹

### ML experiment

In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def train_test() -> None :
    digit = load_digits()
    data = digit.images.reshape((len(digit.images), -1)) # X
    X_train, X_test, y_train, y_test = train_test_split(
        data, digit.target, test_size = 0.2, shuffle = False, random_state = 42
    )
    model = SVC(gamma = 0.001)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f"test accuracy --> {test_acc}")

train_test()

test accuracy --> 0.9583333333333334


### Turning experiments into ML pipelines with zenML

In [5]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[Annotated[np.ndarray, "X_train"], Annotated[np.ndarray, "X_test"], Annotated[np.ndarray, "y_train"], Annotated[np.ndarray, "y_test"]] :
    digits = load_digits()
    data = digits["data"]
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits["target"], test_size = 0.2, shuffle = False, random_state = 42
    )
    return X_train, X_test, y_train, y_test

@step
def svc_trainer(X_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
    model = SVC(gamma = 0.001)
    model.fit(X_train, y_train)
    return model

@step
def evaluator(X_test: np.ndarray, y_test: np.ndarray, model: ClassifierMixin) -> float :
    y_pred = model.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred)
    print(f"test accuracy --> {test_acc}")
    return test_acc


In [6]:
from zenml import pipeline

@pipeline
def digits_pipeline() :
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train = X_train, y_train = y_train)
    evaluator(X_test = X_test, y_test = y_test, model = model)

In [7]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new pipeline: digits_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
No materializer is registered for type <class 'numpy.ndarray'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loaded when running with a different Python version. Please consider implementing a custom materializer for type <class 'numpy.ndarray'> according to the instructions at https://docs.zenml.io/how-to/handle-data-artifacts/handle-custom-data-types
No materializer is registered for type <class 'numpy.ndarray'>, so the default Pickle materializer was used. Pickle is not production ready and should only be used for prototyping as the artifacts cannot be loa

In [8]:
from zenml.environment import Environment

def start_zenml_dashboar(port = 8237) :
    if Environment.in_google_colab() :
        from pyngrok import ngrok
        public_url = ngrok.connect(port)
        print(f"In colab, use this URL instead : {public_url}")
        !zenml up --blocking --port {port}

    else :
        !zenml up --port {port}

start_zenml_dashboar()

The `zenml up` command is deprecated and will be removed in a future release. 
Please use the `zenml login --local` command instead.
Calling `zenml login --local`...
Deploying a local daemon ZenML server.
⠋ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠙ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠹ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠸ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠼ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠴ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠦ Starting service 'DaemonZenServer[9a28cb4f-8e24-43ea-b55b-417e3f15a29a] (type:
zen_server, flavor: daemon)'.
⠧ Starting service

Use default as username and url https://1710-35-196-19-64.ngrok-free.app